# 对于超级富豪的筛选

In [ ]:
import pandas as pd

bill_path = r"C:\Software\Local Things (Coding)\comp\2025大学生建模比赛\数据\all_billionaires_1997_2024.csv"
bill_list = pd.read_csv(bill_path)

bill_list.head()

In [ ]:
"""
Data Wrangler 生成的单元格。
"""
def clean_data(bill_list):
    # 删除列: 'first_name' 中缺少数据的行
    bill_list = bill_list.dropna(subset=['first_name'])
    # 根据列筛选行: 'year'
    bill_list = bill_list[bill_list['year'] > 2019]
    # 按列排序: 'full_name' (升序)
    bill_list = bill_list.sort_values(['full_name'])
    # 根据列筛选行: 'rank'
    bill_list = bill_list[bill_list['rank'] <= 200]
    return bill_list

bill_list_clean = clean_data(bill_list.copy())
bill_list_clean.head()

In [ ]:
value_counts = bill_list_clean['full_name'].value_counts()
value_counts.head()

In [ ]:
# bill_list_final = pd.read_csv(r'bill_list 2019-2024.csv')
bill_list_final = pd.read_excel(r'C:\Software\Local Things (Coding)\comp\2025大学生建模比赛\数据\最终30名富豪名单.xlsx')
name_list = bill_list_final['full_name']
data = bill_list_clean.loc[bill_list_clean['full_name'].isin(name_list)]
# data.head()
data.to_csv('bill_list_final.csv', index=False,encoding='utf-8-sig')

# 处理json文件

## 批量合并json文件

In [ ]:
import os
import json
import pandas as pd

json_folder = r"C:\Users\Fisher Man\Downloads\TweetCrawler"
data = []

# 遍历主文件夹下的所有子文件夹
for person_folder in os.listdir(json_folder):
    person_path = os.path.join(json_folder, person_folder)
    if os.path.isdir(person_path):
        # 遍历子文件夹中的所有JSON文件
        for json_file in os.listdir(person_path):
            if json_file.endswith('.json'):
                json_file_path = os.path.join(person_path, json_file)
                with open(json_file_path, 'r', encoding='utf-8') as f:
                    tweet = json.load(f)
                    # 确保提取的是列表中的第一个元素（如果有的话）
                    if isinstance(tweet, list):
                        tweet = tweet[0]
                    # 提取所需的信息
                    created_at = tweet.get('createdAt')
                    full_text = tweet.get('fullText')
                    # 假设推特作者的名字就是子文件夹的名字
                    data.append([created_at, person_folder, full_text])

# 将提取的数据转换为DataFrame
df = pd.DataFrame(data, columns=['time', 'name', 'full_text'])
# 最后两列需要删去
df.to_csv('data.csv', index=False,encoding='utf-8-sig')


## 合并数据量大的超级富豪

In [ ]:
import pandas as pd
giant5_path = r"C:\Users\Fisher Man\Downloads\TweetCrawler\现有数据集\giant5.xlsx"
small_path = r"C:\Software\Local Things (Coding)\comp\2025大学生建模比赛\数据\data.csv"
elon_path = r"C:\Users\Fisher Man\Downloads\TweetCrawler\现有数据集\all_musk_posts.csv"
giant5 = pd.read_excel(giant5_path)
small = pd.read_csv(small_path)
elon = pd.read_csv(elon_path)
# giant5.head()
print(giant5.columns)
print(small.columns)
print(elon.columns)

print(f"giant5 shape: {giant5.shape} \n small shape: {small.shape} \n elon shape: {elon.shape}")


In [ ]:
key_indicators = ['time', 'name', 'full_text']
giant5.rename(columns={'date': 'time', 'user.username': 'name','rawContent': 'full_text'}, inplace=True)
elon.rename(columns = {'createdAt':'time','fullText': 'full_text'},inplace=True)
elon['name'] = 'elonmusk'
giant5_tbmerge = giant5[key_indicators]
elon_tbmerge = elon[key_indicators]
# timestamp: giant 5:2025-04-04T18:29:29.000Z, elon:2023-05-07 10:36:27+00:00, small:2019-02-12T15:39:52.000Z
print(f'timestamp processing...for giant5')
giant5_tbmerge['time'] = pd.to_datetime(giant5_tbmerge['time']).dt.tz_convert('UTC')
print(f'timestamp processing...for elon')
elon_tbmerge['time'] = pd.to_datetime(elon_tbmerge['time']).dt.tz_convert('UTC')
print(f'timestamp processing...for small')
# small['time'] = pd.to_datetime(small['time']).dt.tz_convert('UTC',format = 'ISO8601')
# small['time'] = pd.to_datetime(small['time'], utc=True)
# small['time'] = pd.to_datetime(small['time'], format='%a %b %d %H:%M:%S %z %Y')
small['time'] = pd.to_datetime(small['time']).dt.tz_convert('UTC')
# print(giant5_tbmerge.head())
print(f"giant5_tbmerge shape: {giant5_tbmerge.shape}")
print(f"giant5_tbmerge columns: {giant5_tbmerge.columns}")
print(f"elon_tbmerge shape: {elon_tbmerge.shape}")
print(f"elon_tbmerge columns: {elon_tbmerge.columns}")
print(f"small columns: {small.columns}")
print(f"small shape: {small.shape}")
# print(f"giant5_tbmerge.dtypes: {giant5_tbmerge.dtypes}")

In [ ]:
merged_df = pd.concat([giant5_tbmerge, elon_tbmerge, small], axis=0)
merged_df.set_index('time', inplace=True)
merged_df.sort_index(inplace=True)

print(merged_df.shape)

merged_df.drop_duplicates(subset='full_text', inplace=True)

print(merged_df.shape)

merged_df.reset_index(inplace=True)
merged_df.head()




In [ ]:
# 定义目标名字列表
target_names = [
    "elonmusk",
    "BillGates",
    "JeffBezos",
    "masason",
    "ericschmidt",
    "RicardoBSalinas",
    "WarrenBuffett",
    "RayDalio",
    "udaykotak",
    "AlikoDangoto",
    "findkd",
    "MichaelDell",
    "JackMa",
    "rupertmurdoch",
    "leijun",
    "carlosslim",
    "StevenACohen2",
    "MukeshDhiAmbani",
    "mackenziescott",  # 贝索斯前妻
    "larryellison",
    "mcannonbrookes",
    "laurenepowell",
    "Steven_Ballmer",
    "scottfarkas",
    "esaverin",
    "lemannoficial",  # 85岁，official号
    "_AmancioOrtega",
    "ShivNadarFDN",
    "Eyalo365"
]

# 打印目标名字列表长度以供检查
print(len(target_names))

# 时间区间
start_time = pd.to_datetime('2019-01-01 00:00:00').tz_localize('UTC')

In [ ]:
merged_df['time'] = pd.to_datetime(merged_df['time'],utc=True)
filtered_df = merged_df[(merged_df['time'] > start_time) & (merged_df['name'].isin(target_names))]
filtered_df.sort_values(by='time', inplace=True)
filtered_df.reset_index(drop=True, inplace=True)
print(filtered_df.shape)
filtered_df.to_csv('filtered_data.csv', index=False)
# print(filtered_df['name'].count())
filtered_df.head()

In [ ]:
"""
Data Wrangler 生成的单元格。
该函数用于统计每个用户名的完整文本出现次数。
"""
def calculate_user_statistics(user_data):
    # 已对列 'name' 执行 1 个聚合 分组
    user_statistics = user_data.groupby(['name']).agg(full_text_count=('full_text', 'count')).reset_index()
    return user_statistics

user_statistics_df = calculate_user_statistics(filtered_df.copy())
user_statistics_df.head()


In [ ]:
"""
Data Wrangler 生成的单元格。
"""
def eyalo_data(filtered_df):
    # 根据列筛选行: 'name'
    filtered_df = filtered_df[filtered_df['name'] == "Eyalo365"]
    return filtered_df

eyalo = eyalo_data(filtered_df.copy())
eyalo.to_csv("eyalo.csv", index=False)
eyalo.head()

## Translation for Eyalo

In [ ]:
import pandas as pd
eyalo = pd.read_csv('eyalo.csv')
eyalo.head()

In [ ]:
eyalo['txt_length'] = eyalo['full_text'].str.len()
total_length = eyalo['txt_length'].sum()
print(f"调用api需要的字符数：{total_length/1000000:.2f}百万")

# 合并指数文件并计算相应的波动率

波动率函数

Step 1:

$$ R_n = ln(\frac{S_n}{S_{n-1}})$$
其中，$S_n$为第n天的收盘价，$S_{n-1}$为第n-1天的收盘价。

Step 2:

$$ R_{avg} = \frac{1}{n}\sum_{i=1}^n R_i $$
we use n = 7 here.

Step 3:

$$ \sigma_R = \sqrt{\frac{1}{n-1}\sum_{i=1}^n (R_i - R_{avg})^2} $$

Step 4:

$$ HV_y = \sigma_R*\sqrt{7} $$ 



In [ ]:
import os
import pandas as pd
import numpy as np

# 设置数据目录
main_path = r"C:\Software\Local Things (Coding)\comp\2025大学生建模比赛\数据\USA index"

# 获取目录下所有 .xlsx 文件
file_list = [f for f in os.listdir(main_path) if f.endswith('.xlsx')]
vol_period = 7  # 计算未来 7 天波动率的周期
dfs = []
for file in file_list:
    file_path = os.path.join(main_path, file)
    # 读取 Excel 文件（默认第一个sheet）
    df = pd.read_excel(file_path)
    
    # 转换“交易日期”为 datetime 类型
    df["交易日期"] = pd.to_datetime(df["交易日期"])
    
    # 处理“收盘价”：去除可能的逗号，并转换为 float
    df["收盘价"] = df["收盘价"].astype(str).str.replace(",", "", regex=False).astype(float)
    
    # 按“交易日期”升序排序
    df = df.sort_values("交易日期").reset_index(drop=True)
    
    # 用于计算未来 7 天波动率的函数，
    # 这里按照公式步骤计算：
    # Step 1: 计算日对数收益率 R_t = ln(S_t / S_{t-1}) （共 7 个返回值，需要 8 天收盘价）
    # Step 2: 计算7个收益率的平均值
    # Step 3: 计算样本标准差 sigma_R = sqrt( sum((R_i-R_avg)^2)/(7-1) )
    # Step 4: HV_y = sigma_R * sqrt(7)
    def calc_hv(idx, price_series):
        # 取从当前日期 idx 开始，连续8天（即当前日及未来7天）的收盘价
        subset = price_series.iloc[idx: idx+vol_period+1]
        if len(subset) < vol_period+1:
            return np.nan
        # 计算日对数收益率，共有7个
        returns = np.log(subset.values[1:] / subset.values[:-1])
        # Step 2: 计算平均收益率
        avg_return = returns.mean()
        # Step 3: 计算样本标准差（自由度为7-1=6）
        sigma_R = np.sqrt(np.sum((returns - avg_return)**2) / (vol_period - 1))
        # Step 4: 波动率调整
        HV_y = sigma_R * np.sqrt(vol_period)
        return HV_y
    
    # 对每个交易日计算未来7天的波动率（计算基于当日到第7天共8天数据）
    df["未来7天波动率"] = [calc_hv(i, df["收盘价"]) for i in range(len(df))]
    
    # 提取文件名中前缀作为指数名称，例如 "DJI" 来自 "DJI.GI-行情统计-20250408.xlsx"
    index_name = file.split('.')[0]
    df["指数"] = index_name
    
    dfs.append(df)

# 合并所有指数的数据
final_df = pd.concat(dfs, ignore_index=True)
final_df = final_df.sort_values(["指数", "交易日期"]).reset_index(drop=True)
final_df.to_csv(r"stock_data.csv", index=False,encoding='utf_8_sig')

In [ ]:
import pandas as pd

# 假设 final_df 已经加载并准备好
df_panel = final_df[['交易日期', '指数', '未来7天波动率']].copy()

# 将 '交易日期' 列转换为时区感知的 datetime 类型
df_panel['交易日期'] = pd.to_datetime(df_panel['交易日期']).dt.tz_localize('UTC')

# 透视数据以创建宽表
df_wide = df_panel.pivot(index='交易日期', columns='指数', values='未来7天波动率')

# 打印宽表的列名
print(df_wide.columns)

# 设置开始时间
start_time = pd.Timestamp('2019-01-01 00:00:00', tz='UTC')

# 过滤数据，保留交易日期大于等于开始时间的记录
df_wide = df_wide[df_wide.index >= start_time]

# 将结果保存为CSV文件
df_wide.to_csv('stock_volatility.csv', index=True,encoding='utf-8-sig')


In [ ]:
import matplotlib.pyplot as plt
import re
import numpy as np

# 原始数据解析
log_data = """
[您的完整日志数据粘贴在此]
"""

# 数据提取模式
epoch_pattern = r"Epoch (\d+)/(\d+).*train_acc=([\d.]+), val_acc=([\d.]+)"
batches = re.findall(epoch_pattern, log_data)

# 数据结构化处理
training_records = []
for epoch, total_epochs, train_acc, val_acc in batches:
    training_records.append({
        "epoch": int(epoch),
        "total_epochs": int(total_epochs),
        "train_acc": float(train_acc),
        "val_acc": float(val_acc)
    })

# 关键阶段划分
phases = {
    "activation": slice(0, 25),       # 前5次5 epoch实验
    "optimizer": slice(25, 45),       # 后续4次5 epoch实验
    "epoch_tuning": slice(45, 105),   # 不同epoch数实验
    "batch_tuning": slice(105, 165),  # 批量大小实验
    "final_tuning": slice(165, None)  # 最终调整
}

# 可视化设置
plt.figure(figsize=(15, 18))
plt.suptitle("Model Training Analysis", y=1.02)

# 子图1: 验证准确率发展趋势
ax1 = plt.subplot(311)
val_accs = [x["val_acc"] for x in training_records]
ax1.plot(val_accs, 'b-', alpha=0.6, label="Validation Accuracy")
ax1.scatter(np.argmax(val_accs), max(val_accs), c='r', 
          label=f"Best val_acc: {max(val_accs):.4f}")
ax1.set_ylabel("Accuracy")
ax1.set_xlabel("Training Progress")
ax1.legend()
ax1.grid(True)

# 子图2: 阶段验证准确率分布
ax2 = plt.subplot(312)
phase_labels = ["Activation", "Optimizer", "Epoch", "Batch", "Final"]
phase_accs = [
    np.mean([x["val_acc"] for x in training_records[phases["activation"]]]),
    np.mean([x["val_acc"] for x in training_records[phases["optimizer"]]]),
    np.mean([x["val_acc"] for x in training_records[phases["epoch_tuning"]]]),
    np.mean([x["val_acc"] for x in training_records[phases["batch_tuning"]]]),
    np.mean([x["val_acc"] for x in training_records[phases["final_tuning"]]])
]
ax2.bar(phase_labels, phase_accs, alpha=0.6, 
       color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd'])
ax2.set_ylabel("Average Val Accuracy")
ax2.set_title("Phase Performance Comparison")
ax2.grid(True)

# 子图3: 训练/验证曲线
ax3 = plt.subplot(313)
train_accs = [x["train_acc"] for x in training_records]
ax3.plot(train_accs, 'g--', label="Training Accuracy")
ax3.plot(val_accs, 'b-', label="Validation Accuracy")
ax3.set_ylabel("Accuracy")
ax3.set_xlabel("Epochs")
ax3.legend()
ax3.grid(True)

plt.tight_layout()
plt.savefig("training_analysis.png", dpi=300, bbox_inches='tight')
plt.show()

# 输出最终测试结果
test_acc = 0.6705  # 从日志中提取的测试准确率
print(f"Final Test Accuracy: {test_acc:.2%}")
